In [16]:
import pandas as pd
import numpy as np
from time import time
import json
from sqlalchemy import create_engine
import re
import datetime
#import lifetimes
#import pymc3 as pm

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

In [17]:
# Set up the SQL connection
dbname = 'redshift'
url = 'panamera-ods.cnsuxis6zqxr.us-west-2.redshift.amazonaws.com:5439/panameraods'
engine = create_engine("postgresql://" + 'lucasdresl' + ":" + 'cQsY801a76x82x62aWDT' + "@" + url)

# Colin

In [66]:
query = """

select
  extract(month from date_event_nk) as month,
  channel_sk,
  split_part(country_sk , '|' , 1)||split_part(country_sk , '|' , 3) as country,
  login_method,
  user_nk
    from ods.fact_login_complete
 where country_sk in ('olx|asia|in' , 'olx|asia|id')
 and date_event_nk >= '2019-08-20' and date_event_nk<='2019-08-31'
 and login_method in ('google' , 'facebook' , 'phone' , 'email')
group by 1,2,3,4,5
order by 5


"""

In [67]:
df = pd.read_sql_query(query.replace('%', '%%'), engine)

In [68]:
df.head()

,month,channel_sk,country,login_method,user_nk
0,8,web|desktop_web,olxid,email,48.0
1,8,web|desktop_web,olxid,email,61.0
2,8,web|desktop_web,olxid,email,77.0
3,8,mobile_app|ios,olxid,email,120.0
4,8,mobile_app|android,olxid,email,120.0


In [69]:
df = df[df['user_nk'].notnull()]

In [70]:
dfid = df[df['country'] == 'olxid'].sample(n=50000)
dfin = df[df['country'] == 'olxin'].sample(n=50000)
dfin.user_nk = dfin.user_nk.astype(int)
dfid.user_nk = dfid.user_nk.astype(int)

In [71]:
dfid.head()

,month,channel_sk,country,login_method,user_nk
1600647,8,mobile_app|android,olxid,email,100384734
1278248,8,web|mobile_web,olxid,google,100060144
1467129,8,mobile_app|android,olxid,facebook,100249778
1042004,8,mobile_app|android,olxid,google,93247297
1565369,8,mobile_app|android,olxid,facebook,100348771


In [72]:
# Connection to the SQL database
import pymysql.cursors
connect_sql = pymysql.connect(host='prd-db-olxin-replica-1.cnhuszkczyjo.ap-southeast-1.rds.amazonaws.com',
                            user='dev',
                            password='^b6W2vVuM4KJ',
                            db='olxin',
                            charset='utf8mb4',
                            cursorclass=pymysql.cursors.DictCursor)

In [73]:
query = """

SELECT
 id,
 case when facebook_id is not null then 1 else 0 end as has_facebook,
 case when google_id is not null then 1 else 0 end as has_google,
 case when default_phone is not null then 1 else 0 end as has_phone,
 case
    WHEN email LIKE '%@olx%' THEN 0 -- Autogenerado
    WHEN email LIKE '%DELETED%' THEN 0 -- Autogenerados
    WHEN email LIKE '%UNLINK%' THEN 0 -- Autogenerados
    ELSE 1 END  as valido_email
FROM olxin.users
    where id in {} 

""".format('(' + ', '.join(pd.Series(dfin.user_nk).astype('str').apply(lambda x: "'"  + x + "'")) + ')')

In [74]:
df_ora = pd.read_sql(query, con=connect_sql)

/Users/lucas/anaconda3/lib/python3.7/site-packages/pymysql/cursors.py:329: Warning: (3170, "Memory capacity of 8388608 bytes for 'range_optimizer_max_mem_size' exceeded. Range optimization was not done for this query.")
  self._do_get_result()


In [75]:
df_ora.rename(columns={'id' : 'user_nk'} , inplace=True)
#dfid.rename(columns={'id' : 'user_nk'} , inplace=True)

In [76]:
dfin_c = pd.merge(dfin , df_ora , how='left' , on=['user_nk']).fillna(0)
#dfid_c = pd.merge(dfid , df_ora , how='left' , on=['user_nk']).fillna(0)

In [77]:
dfin_c.head()

,month,channel_sk,country,login_method,user_nk,has_facebook,has_google,has_phone,valido_email
0,8,mobile_app|android,olxin,facebook,424838816,1,0,1,0
1,8,mobile_app|android,olxin,google,308164991,1,1,1,1
2,8,mobile_app|android,olxin,phone,425654671,0,0,1,0
3,8,web|desktop_web,olxin,google,271708237,0,1,0,1
4,8,mobile_app|android,olxin,facebook,44585481,1,0,0,1


In [79]:
dfin_c.pivot_table(index='channel_sk' ,columns='login_method' , values='user_nk' , aggfunc={'user_nk':'count' ,
                                                                                           'user_nk':pd.Series.nunique})

login_method,email,facebook,google,phone
channel_sk,,,,
mobile_app|android,1241,7892,13380,14278
mobile_app|ios,73,322,304,658
web|desktop_web,337,248,1182,1793
web|mobile_web,278,581,4641,2792


In [81]:
dfin_c.pivot_table(index='channel_sk' , values='user_nk' , aggfunc={'user_nk':pd.Series.nunique})                                               

,user_nk
channel_sk,
mobile_app|android,36781
mobile_app|ios,1357
web|desktop_web,3560
web|mobile_web,8288


In [82]:
dfin_c.pivot_table(index='channel_sk' , values=['has_facebook' , 'has_google' , 'has_phone','valido_email','user_nk'] ,
                  aggfunc={'has_facebook':sum , 'has_google':sum , 'has_phone':sum , 'valido_email':sum ,
                          'user_nk':pd.Series.nunique})

,has_facebook,has_google,has_phone,user_nk,valido_email
channel_sk,,,,,
mobile_app|android,14128,18005,28869,36781,20104
mobile_app|ios,532,491,1099,1357,611
web|desktop_web,1799,2359,3191,3560,2019
web|mobile_web,5157,6671,6968,8288,5514


In [83]:
dfin_notface = dfin_c[dfin_c['login_method'] != 'facebook']

In [86]:
dfin_notface['has_facebook'].value_counts()

0    28276
1    12681
Name: has_facebook, dtype: int64

In [87]:
# Connection to the SQL database
import pymysql.cursors
connect_sql = pymysql.connect(host='prd-db-olxid-replica-1.cnhuszkczyjo.ap-southeast-1.rds.amazonaws.com',
                            user='dev',
                            password='^b6W2vVuM4KJ',
                            db='olxid',
                            charset='utf8mb4',
                            cursorclass=pymysql.cursors.DictCursor)

In [88]:
query = """

SELECT
 id,
 case when facebook_id is not null then 1 else 0 end as has_facebook,
 case when google_id is not null then 1 else 0 end as has_google,
 case when default_phone is not null then 1 else 0 end as has_phone,
 case
    WHEN email LIKE '%@olx%' THEN 0 -- Autogenerado
    WHEN email LIKE '%DELETED%' THEN 0 -- Autogenerados
    WHEN email LIKE '%UNLINK%' THEN 0 -- Autogenerados
    ELSE 1 END  as valido_email
FROM olxid.users
    where id in {} 

""".format('(' + ', '.join(pd.Series(dfid.user_nk).astype('str').apply(lambda x: "'"  + x + "'")) + ')')

In [89]:
df_ora = pd.read_sql(query, con=connect_sql)

/Users/lucas/anaconda3/lib/python3.7/site-packages/pymysql/cursors.py:329: Warning: (3170, "Memory capacity of 8388608 bytes for 'range_optimizer_max_mem_size' exceeded. Range optimization was not done for this query.")
  self._do_get_result()


In [90]:
df_ora.rename(columns={'id' : 'user_nk'} , inplace=True)
#dfid.rename(columns={'id' : 'user_nk'} , inplace=True)

In [91]:
dfid_c = pd.merge(dfid , df_ora , how='left' , on=['user_nk']).fillna(0)
#dfid_c = pd.merge(dfid , df_ora , how='left' , on=['user_nk']).fillna(0)

In [92]:
dfid_c.head()

,month,channel_sk,country,login_method,user_nk,has_facebook,has_google,has_phone,valido_email
0,8,mobile_app|android,olxid,email,100384734,0.0,0.0,1.0,1.0
1,8,web|mobile_web,olxid,google,100060144,1.0,1.0,1.0,1.0
2,8,mobile_app|android,olxid,facebook,100249778,1.0,0.0,0.0,0.0
3,8,mobile_app|android,olxid,google,93247297,0.0,1.0,0.0,1.0
4,8,mobile_app|android,olxid,facebook,100348771,1.0,0.0,0.0,0.0


In [93]:
dfid_c.pivot_table(index='channel_sk' ,columns='login_method' , values='user_nk' , aggfunc={'user_nk':'count' ,
                                                                                           'user_nk':pd.Series.nunique})

login_method,email,facebook,google,phone
channel_sk,,,,
mobile_app|android,4000.0,7722.0,13333.0,3.0
mobile_app|ios,643.0,1042.0,1441.0,NaN
web|desktop_web,1545.0,635.0,2338.0,NaN
web|mobile_web,1959.0,2466.0,12872.0,1.0


In [94]:
dfid_c.pivot_table(index='channel_sk' , values='user_nk' , aggfunc={'user_nk':pd.Series.nunique})                                               

,user_nk
channel_sk,
mobile_app|android,25047
mobile_app|ios,3124
web|desktop_web,4505
web|mobile_web,17286


In [96]:
dfid_c.pivot_table(index='channel_sk' , values=['has_facebook' , 'has_google' , 'has_phone','valido_email','user_nk'] ,
                  aggfunc={'has_facebook':sum , 'has_google':sum , 'has_phone':sum , 'valido_email':sum ,
                          'user_nk':pd.Series.nunique})

,has_facebook,has_google,has_phone,user_nk,valido_email
channel_sk,,,,,
mobile_app|android,9829.0,14212.0,17243.0,25047,23538.0
mobile_app|ios,1288.0,1556.0,1980.0,3124,3043.0
web|desktop_web,2018.0,3053.0,3946.0,4505,4381.0
web|mobile_web,11836.0,15106.0,14937.0,17286,16522.0


In [99]:
dfid_notface = dfid_c[dfid_c['login_method'] != 'facebook']

In [100]:
dfid_notface.shape

(38135, 9)

In [101]:
dfid_notface['has_facebook'].value_counts()

0.0    24806
1.0    13329
Name: has_facebook, dtype: int64